In [ ]:
import sys, os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

data_dir = os.path.join(os.getcwd(), "data_1")
import pcse

In [ ]:
data_dir = os.path.join(os.path.dirname(os.path.join(os.getcwd())), "nevergrad", "functions","irrigation","data","data_1")
print(data_dir)


##### Paramètre de culture

In [ ]:
from pcse.fileinput import CABOFileReader
cropfile = os.path.join(data_dir, 'crop', 'SUG0601.crop')
cropd = CABOFileReader(cropfile)

In [ ]:
#Paramètre du sol
soilfile = os.path.join(data_dir, 'soil', 'ec3.soil')
soild = CABOFileReader(soilfile)

In [ ]:
#Paramètre de site
from pcse.util import WOFOST72SiteDataProvider
sited = WOFOST72SiteDataProvider(WAV=10, CO2=360)
print(sited)

In [ ]:
#Mise en commun des paramètres
from pcse.base import ParameterProvider
parameters = ParameterProvider(cropdata=cropd, soildata=soild, sitedata=sited)

In [ ]:
#agrogestion
from pcse.fileinput import YAMLAgroManagementReader
agromanagement_file = os.path.join(data_dir, 'agro', 'sugarbeet_calendar.agro')
agromanagement = YAMLAgroManagementReader(agromanagement_file)
print(agromanagement)

#agrogestion avec irrigation
agromanagement_file_irrigation = os.path.join(data_dir, 'agro', 'sugarbeet_calendar_irrigation.agro')
agromanagement_irrigation = YAMLAgroManagementReader(agromanagement_file_irrigation)
print(agromanagement_irrigation)

In [ ]:
#Données météorologigues
from pcse.fileinput import ExcelWeatherDataProvider
weatherfile = os.path.join(data_dir, 'meteo', 'nl1.xlsx')
wdp = ExcelWeatherDataProvider(weatherfile)
print(wdp)



In [ ]:
#construction du modèle et initialisation
from pcse.models import Wofost72_WLP_FD, Wofost72_PP
wofsim = Wofost72_WLP_FD(parameters, wdp, agromanagement)
wofsim_irrigation = Wofost72_WLP_FD(parameters, wdp, agromanagement_irrigation)

In [ ]:
#lancement de la simu
wofsim.run_till_terminate()
wofsim_irrigation.run_till_terminate()
df_results = pd.DataFrame(wofsim.get_output())
df_results = df_results.set_index("day")
df_results.tail()

In [ ]:
df1_results = pd.DataFrame(wofsim_irrigation.get_output())
df1_results = df1_results.set_index("day")
df1_results.tail()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12,10))
for var, ax in zip(["DVS", "TAGP", "LAI", "SM"], axes.flatten()):
    ax.plot_date(df_results.index, df_results[var], 'b-')
    ax.set_title(var)
    ax.grid()
fig.autofmt_xdate()

plt.savefig('courbe_avant_irrigation.png')

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12,10))
for var, ax in zip(["DVS", "TAGP", "LAI", "SM"], axes.flatten()):
    ax.plot_date(df1_results.index, df1_results[var], 'b-')
    ax.set_title(var)
    ax.grid()
fig.autofmt_xdate()
plt.savefig('courbe_after_irrigation.png')